In [6]:
import pandas as pd
import tensorflow as tf
import numpy as np
import os
import datetime
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from ipynb.fs.full.Preprocessing import clean_data
%load_ext tensorboard

df = pd.read_csv('./Data/songs_cleaned.csv', index_col='Unnamed: 0')
print(df.shape)
df.describe()

/opt/anaconda3/envs/Unit_4/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Old df shape: (586672, 20)
Old df columns: ['id', 'name', 'popularity', 'duration_ms', 'explicit', 'artists', 'id_artists', 'release_date', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']

Cleaned df shape: (586344, 26)
Clean df columns: ['popularity', 'duration_ms', 'explicit', 'danceability', 'energy', 'key_1', 'key_2', 'key_3', 'key_4', 'key_5', 'key_6', 'key_7', 'key_8', 'key_9', 'key_10', 'key_11', 'key_12', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']
(586344, 26)


,popularity,duration_ms,explicit,danceability,energy,key_1,key_2,key_3,key_4,key_5,...,key_12,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
count,5.863440e+05,5.863440e+05,5.863440e+05,5.863440e+05,5.863440e+05,586344.000000,586344.000000,586344.000000,586344.000000,586344.000000,...,586344.000000,5.863440e+05,586344.000000,5.863440e+05,5.863440e+05,5.863440e+05,5.863440e+05,5.863440e+05,5.863440e+05,5.863440e+05
mean,-6.541704e-14,4.196940e-16,-1.393941e-13,-5.074522e-15,-2.380274e-15,0.127717,0.071086,0.125781,0.036711,0.091388,...,0.111044,2.978384e-16,0.658792,-2.960057e-15,-1.125049e-14,-4.119535e-14,8.449369e-17,-1.382204e-16,7.580292e-16,9.434782e-14
std,9.998227e-01,9.952174e-01,1.000212e+00,9.970543e-01,9.994280e-01,0.333775,0.256969,0.331603,0.188050,0.288161,...,0.314187,9.963881e-01,0.474115,1.000185e+00,9.998803e-01,9.992351e-01,9.996187e-01,9.989943e-01,9.958386e-01,9.813526e-01
min,-1.500701e+00,-1.697285e+00,-2.148490e-01,-3.074662e+00,-2.152405e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-8.802915e+00,0.000000,-5.832490e-01,-1.289455e+00,-4.244867e-01,-1.129392e+00,-2.144603e+00,-2.957428e+00,-8.190745e+00
25%,-7.930492e-01,-4.340974e-01,-2.148490e-01,-6.677147e-01,-7.869069e-01,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-5.279452e-01,0.000000,-3.942474e-01,-1.011674e+00,-4.244867e-01,-6.271283e-01,-8.018040e-01,-7.687832e-01,2.630195e-01
50%,-3.096308e-02,-1.195375e-01,-2.148490e-01,7.881222e-02,2.683466e-02,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.884565e-01,1.000000,-3.369910e-01,-7.971846e-02,-4.243953e-01,-4.063232e-01,4.423698e-02,-3.746548e-02,2.630195e-01
75%,7.311231e-01,2.673159e-01,-2.148490e-01,7.350335e-01,8.167594e-01,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,7.307683e-01,1.000000,-1.585512e-01,9.580173e-01,-3.888886e-01,3.477778e-01,8.398260e-01,5.982336e-01,2.630195e-01
max,3.942772e+00,4.260912e+01,4.656401e+00,2.571249e+00,1.817066e+00,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,3.060351e+00,1.000000,4.814415e+00,1.565752e+00,3.322689e+00,4.264763e+00,1.736319e+00,4.295441e+00,2.376461e+00


In [ ]:
# Simple autoencoder

encoded_dim = 16

input_song = Input(shape = (df.shape[1], ))

h1 = Dense(32, activation = 'relu')(input_song)

encoded = Dense(encoded_dim, activation = 'relu')(h1)

dh1 = Dense(32, activation='sigmoid')(encoded)

decoded = Dense(df.shape[1], activation='sigmoid')(dh1)

autoencoder = Model(input_song, decoded)

encoder = Model(input_song, encoded)

autoencoder.compile(optimizer='nadam', loss='mean_squared_error')

In [ ]:
stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3)
now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

logdir = os.path.join("logs", f"SimpleAutoencoder-{now}")
tensorboard = TensorBoard(log_dir=logdir)

autoencoder.fit(df, # input image to encoder
                df, # provide input image to decoder so the model learns how to reconstruct the input image 
                epochs=100,
                validation_split=.2,
                callbacks=[stop, tensorboard])

### Shallow NN Summary:
0 hidden layers: Val Loss: 0.3160
1 hidden layer: 

In [ ]:
decoded_songs = autoencoder.predict(df)
encoded_songs = encoder.predict(df)

embeddings_df = pd.DataFrame(encoded_songs, index=df.index)

In [ ]:
embeddings_df

In [ ]:
raw_df

In [ ]:
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=11, algorithm='ball_tree')
nn.fit(encoded_songs)
nn

In [ ]:
# Saving the encoded_songs embedding as a df and the Nearest Neighbours model for use in our app
import pickle

embeddings_df = pd.DataFrame(encoded_songs, index=df.index)

# Saving the embeddings to the disk
embeddings_df.to_csv('./saved_models/embeddings_df_001.csv')

# save the model to disk
filename = './saved_models/nearestneigh_001.sav'
pickle.dump(nn, open(filename, 'wb'))

# some time later...
 
# load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)
#print(result)

In [ ]:
# # testing with Beautiful Day - U2

# raw_df = pd.read_csv('./Data/tracks.csv')
# test_songs = {'Beautiful Day': '0gzqZ9d1jIKo9psEIthwXe'}
# for name, id in test_songs.items():
#     query = raw_df.index[raw_df['id'] == '0gzqZ9d1jIKo9psEIthwXe'][0]
#     test_song = pd.DataFrame([raw_df.iloc[query, :]])

#     score, close_image_idx = nn.kneighbors(encoded_songs[query].reshape(1,-1))
#     k_closest_songs_indexes = close_image_idx[0][1:]

#     print('Top 10 recommendations for {} by {}:\n'.format(name, test_song.artists))
#     for i in range(len(k_closest_songs_indexes)):
#         song = raw_df.iloc[k_closest_songs_indexes[i], :]
#         print('{} by {} with a score of {}\n'.format(song['name'], song.artists, score[0][1:][i]))
#         print('URL: https://open.spotify.com/track/{}\n'.format(song['id']))
#     print('___________________________________________________________________________________\n')


In [ ]:
# Testing our model with multiple songs
raw_df = pd.read_csv('./Data/tracks.csv')

test_songs = {'California Love': '0LyLz6XsVs6wz85dK0S6EG', 'The Gambler': '5KqldkCunQ2rWxruMEtGh0', 
              'Eye of the Tiger': '2HHtWyy5CgaQbC7XSoOb0e', 'You are my Sunshine': '4ApTr4wbyOucy2lbAvyuuV'}

# Loop over all our test songs and get the recommendations
recommendations = {}

print('DeepSpot\'s recommendation system:\n\n')

for name, id in test_songs.items():
    query = raw_df.index[raw_df['id'] == id][0]
    test_song = pd.DataFrame([raw_df.iloc[query, :]])
    
    score, close_image_idx = nn.kneighbors(encoded_songs[query].reshape(1,-1))
    k_closest_songs_indexes = close_image_idx[0][1:]
    
    print('Top 10 recommendations for {} by {}:\n'.format(name, test_song.artists))
    for i in range(len(k_closest_songs_indexes)):
        song = raw_df.iloc[k_closest_songs_indexes[i], :]
        print('{} by {} with a score of {}\n'.format(song['name'], song.artists, score[0][i]))
        print('URL: https://open.spotify.com/track/{}\n'.format(song['id']))
    print('___________________________________________________________________________________\n')
    
    recommendations[name] = k_closest_songs_indexes


In [ ]:
# Deep variational encoder

encoded_dim = 16

input_song = Input(shape = (df.shape[1], ))

encoded_h1 = Dense(64, activation = 'relu')(input_song)
encoded_h2 = Dense(32, activation = 'relu')(encoded_h1)
encoded = Dense(encoded_dim, activation = 'relu')(encoded_h2)

decoded_h1 = Dense(32, activation='relu')(encoded)
decoded_h2 = Dense(64, activation='relu')(decoded_h1)
decoded = Dense(df.shape[1], activation='sigmoid')(decoded_h2)

deep_autoencoder = Model(input_song, decoded) # techincally autoencoder_and_decoder

deep_encoder = Model(input_song, encoded) # Half of the autoencoder

deep_autoencoder.compile(optimizer='nadam', loss='mean_squared_error')

In [ ]:
stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3)
now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

logdir = os.path.join("logs", f"SimpleAutoencoder-{now}")
tensorboard = TensorBoard(log_dir=logdir)

deep_autoencoder.fit(df, # input image to encoder
                df, # provide input image to decoder so the model learns how to reconstruct the input image 
                epochs=100,
                validation_split=.2,
                callbacks=[stop, tensorboard])

### Deep NN Summary
Val loss = 0.2960

In [ ]:
decoded_songs = autoencoder.predict(df)
encoded_songs = encoder.predict(df)

from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=11, algorithm='ball_tree')
nn.fit(encoded_songs)

In [ ]:
encoded_songs.shape

In [ ]:
# # Testing
# raw_df = pd.read_csv('./Data/tracks.csv')

# test_songs = {'California Love': '0LyLz6XsVs6wz85dK0S6EG', 'The Gambler': '5KqldkCunQ2rWxruMEtGh0', 
#               'Eye of the Tiger': '2HHtWyy5CgaQbC7XSoOb0e', 'You are my Sunshine': '4ApTr4wbyOucy2lbAvyuuV'}

# # Loop over all our test songs and get the recommendations
# recommendations = {}

# print('DeepSpot\'s recommendation system:\n\n')

# for name, id in test_songs.items():
#     query = raw_df.index[raw_df['id'] == id][0]
#     test_song = pd.DataFrame([raw_df.iloc[query, :]])
    
#     score, close_image_idx = nn.kneighbors(encoded_songs[query].reshape(1,-1))
#     k_closest_songs_indexes = close_image_idx[0][1:]
    
#     print('Top 10 recommendations for {} by {}:\n'.format(name, test_song.artists))
#     for i in range(len(k_closest_songs_indexes)):
#         song = raw_df.iloc[k_closest_songs_indexes[i], :]
#         print('{} by {} with a score of {}\n'.format(song['name'], song.artists, score[0][i]))
#         print('URL: https://open.spotify.com/track/{}\n'.format(song['id']))
#     print('___________________________________________________________________________________\n')
    
#     recommendations[name] = k_closest_songs_indexes



# Training a Variational Autoencoder

In [ ]:
# Variational Autoencoder
from keras import layers

class Sampling(layers.Layer):
    """
    Uses (z_mean, z_log_var) to sample a z vector from the latent N-dimensional Normal distribution
    i.e. Z is the vector encoding a digit.
    """
    def call(self, inputs):
        
        # recall from the video that z_mean and z_log_var are vectors 
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        
        # sample from an N-dimensional normal distribution
        # epsilon is given shape (batch, dim) because we are adding it to z_mean which has shape (batch, dim)
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        
        # output of tf.exp() is a vector 
        simga = tf.exp(0.5 * z_log_var)
        
        # this is our hideen latent vector made up of a mean and variance vector 
        # variance vector is scaled by epsilon, which is sampled from a normal distribtuion 
        # the the video guy said "stocastic" in reference to epsilon, he meant random 
        Z = z_mean + simga * epsilon
        
        # return hidden latent vector 
        return Z

In [ ]:
# recall from the video that the more dimensions that our latent vector has
# the better the results of our model 
latent_dim = 16

# shape of our input data
# we are creating our input layer using Keras's Input() class
# the only thing that input layers really do is define the dimensionality of the input data for the model
encoder_inputs = Input(shape=(df.shape[1], ))

# these are the hidden layers

# pass data vector into FCFF layer 
encoded = Dense(encoded_dim, activation = 'relu')(encoder_inputs)

# recall that ordinarly the output of the last encoding layer is the latent vector 
# but here we are creating two output layers for our encoder - one for the mean and one for the log variance 
# returns a 2-dim mean vector
z_mean = Dense(latent_dim, name="z_mean")(encoded)
# returns a 2-dim log variance vecotr 
z_log_var = Dense(latent_dim, name="z_log_var")(encoded)
# pass mean and variance vector into Sampling class in order to create the Z vector,  Z = mean + var * epsilon
z = Sampling()([z_mean, z_log_var])

# ok, now let's put it all together 
# this is our encoder model 
# inputs are the original images
# outputs are the Z vectors: mean, log variance, and the complete Z, i.e. vector Z = mean + var * epsilon
encoder = Model(inputs=encoder_inputs, outputs=[z_mean, z_log_var, z], name="encoder")
encoder.summary()

In [ ]:
# the input layer to our decoder has the same dimensionality as the latent vector
# because the latent vector is the input to the decoder model
latent_inputs = Input(shape=(latent_dim,))

# these are the hidden layers of our decoder 
# the data at this point is in a vector, the Z latent vector 

# this is the final layer in out decoder
# therefore this layer outputs the reconstruction of the original image 
decoder_outputs = Dense(df.shape[1], activation='sigmoid')(latent_inputs)

# this is our decoder model 
decoder = Model(inputs=latent_inputs, outputs=decoder_outputs, name="decoder")
decoder.summary()

In [ ]:
from tensorflow import keras

class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        """
        This class build a Variational Auto-Encoder. It accepts an Encoder and Decoder model as input. 
        
        Note
        ----
        This VAE class is inheriting Keras's Model API so that it can use the Model class methods 

        """
        # how python 3 handels inheritance 
        super(VAE, self).__init__(**kwargs)
        # set encoder model as class attribute
        self.encoder = encoder
        # set decoder model as class attribute 
        self.decoder = decoder
        # set mean function as class attribute - this calculates the total loss
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        # set mean function as class attribute - this calculates the reconstruction loss
        self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
        # set mean function as class attribute - this calculates the kl loss
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        """
        Returns all loses in a list
        """
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        """
        Training our model via gradient descent and back-propagation 
        """
        
        # we used tf.GradientTape() in Sprint 2 Module 2 to run Gradient Descent from scratch 
        with tf.GradientTape() as tape:
            # pass input data into encoder model 
            # output of encoder model is the hidden state distribution parameters and hidden state vector 
            z_mean, z_log_var, z = self.encoder(data)
            
            # pass hidden state vector into decoder model 
            reconstruction = self.decoder(z)
            
            # calculate the reconstruction loss 
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.MeanSquaredError(data, reconstruction)#, axis=(1, 2)
                )
            )
            
            # calculate the kl loss
            #                (1 + z_simga   - (z_mean)^2        - e^(z_simga) ) 
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            # recall that we used tf.reduce_sum() in Sprint 2 Module 4 assignment 
            # it takes the sum of the vector components 
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            
            # calculate the total loss by adding the two loss components 
            total_loss = reconstruction_loss + kl_loss
            
        # now that we have calculated the loss function, we can perform Gradient Descent
        # we are passing in the loss function and the weights that we want to update via Gradeint Descent 
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        
        # log the total loss
        self.total_loss_tracker.update_state(total_loss)
        
        # log the reconsgrution loss 
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        
        # log the kl loss 
        self.kl_loss_tracker.update_state(kl_loss)
        
        # return all the losses in a dictionary 
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [ ]:
# instantiate a Variational Auto-Encoder model 
vae = VAE(encoder, decoder)

# complie the model 
vae.compile(optimizer=keras.optimizers.Nadam())

# train the model weights 
vae.fit(df, epochs=10, workers=10)
# if you have access to multiple processors, set parameter `workers` to N - 1
# where N is the total number of processors that you have 

In [ ]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
autoencoder.save('saved_model/autoencoder_002')
encoder.save('saved_model/encoder_002')